## Methods for Accessing HEASARC Data  
In this notebook, we will explore various methods to access and download data from the HEASARC (High Energy Astrophysics Science Archive Research Center) archive. While our focus will be on Swift satellite data, the approaches demonstrated here can be applied to data from any HEASARC-hosted mission. The methods covered in this notebook are adapted from the [sciserver_cookbooks](https://github.com/HEASARC/sciserver_cookbooks) GitHub repository.  

### 1. Online Access via Swift Master Catalog
The Swift Master Catalog is a web-based interface that allows users to browse and download Swift mission data. It provides a user-friendly way to search for observations, view metadata, and download datasets directly through the HEASARC website. This method is ideal for quick, manual queries or exploring the data interactively.  
[Access the Swift Master Catalog](https://heasarc.gsfc.nasa.gov/w3browse/swift/swiftmastr.html)

### 2. HEASARC TAP Service (Table Access Protocol)

The TAP service is a standardized Virtual Observatory protocol that enables programmatic access to astronomical catalogs and data tables. Through TAP, users can execute SQL-like queries to retrieve data from the HEASARC archives in a flexible and automated manner. This is particularly useful for large-scale data mining, batch queries, or integrating HEASARC data access into software pipelines.  
Learn more: [TAP Service](https://www.ivoa.net/documents/TAP/20190626/PR-TAP-1.1-20190626.html), [HEASARC API](https://heasarc.gsfc.nasa.gov/docs/archive/apis.html)

### 3. Xamin's API
Xamin is an advanced query and data mining tool provided by HEASARC that supports complex data searches and cross-matching across multiple catalogs. Its API allows users to submit queries programmatically and retrieve data in various formats. Xamin’s capabilities make it a powerful resource for in-depth data exploration and multi-wavelength studies.  
[Xamin](https://heasarc.gsfc.nasa.gov/xamin/)

## Module Imports

In [1]:
import sys
import os
import pyvo
from astropy.coordinates import SkyCoord
import requests
import glob
import numpy as np

## HEASARC TAP service

We can use the Virtual Observatory interfaces to the HEASARC to find the data we're interested in. 

In [22]:
tap_services = pyvo.regsearch(servicetype='tap', keywords=['heasarc'])

In [23]:
heasarc_tables = tap_services[0].service.tables

In [ ]:
for tablename in heasarc_tables.keys():
    if "swift" in tablename:
        print(" {:20s} {}".format(tablename, heasarc_tables[tablename].description))

In [ ]:
for column in heasarc_tables['swiftmastr'].columns:
    print("{:20s} {}".format(column.name, column.description))

In [ ]:
# Get the coordinate for Eta Car
pos = SkyCoord.from_name("OJ 287")
query = """SELECT name, obsid, ra, dec, start_time, processing_date, xrt_exposure, uvot_exposure, bat_exposure, archive_date, target_id, xrt_expo_wt, xrt_expo_pc, uvot_expo_uu, uvot_expo_bb, uvot_expo_vv, uvot_expo_w1, uvot_expo_w2, uvot_expo_m2 
    FROM public.swiftmastr as cat 
    where 
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{},{},0.1))=1 
    and 
    cat.xrt_exposure > 0 order by cat.start_time
    """.format(pos.ra.deg, pos.dec.deg)

In [ ]:
results = tap_services[0].search(query).to_table()
results

In [ ]:
import pandas as pd
from astropy.table import Table
obs_to_check = [res for res in results if 58932 <= res['start_time'] <= 58993]
# filtered_table = Table(rows=obs_to_check)
# filtered_table
# column_names = results.colnames
# df = pd.DataFrame(obs_to_check, columns=column_names)
# df

## Xamin's API

In [ ]:
url = "https://heasarc.gsfc.nasa.gov/xamin/QueryServlet?products&"
params = {
    "table": "swiftmastr",
    "object": "OJ 287",
    "resultmax": "10000"
}

result = requests.get(url, params)
result.text.split('\n')[0:2]

## Finding and Downloading the data

In [2]:
# Create a cone-search service
nu_services = pyvo.regsearch(ivoid='ivo://nasa.heasarc/swiftmastr')[0]
cs_service = nu_services.get_service('conesearch')

In [3]:
# Find the coordinates of the source
pos = SkyCoord.from_name('Mrk 501')

search_result = cs_service.search(pos, verbosity=3)
table = search_result.to_table()
table

DALServiceError: 502 Server Error: Bad Gateway for url: https://heasarc.gsfc.nasa.gov/xamin/vo/cone?showoffsets&table=swiftmastr&&RA=253.46756952&DEC=39.76016913&SR=1.0&VERB=3

In [33]:
# # Find the coordinates of the source
# pos = SkyCoord.from_name('OJ 287')

# search_result = cs_service.search(pos)

# # display the result as an astropy table
# search_result.to_table()

In [45]:
# obs_to_explore = [res for res in search_result if 58932 <= res['start_time'] <= 58993 and res['xrt_exposure'] > 1000]
# obs_to_explore

[('266222', 'OJ287', '00035905040', '133.66536', '20.08646', '58950.9976', '58961.0', '1170.91', '1144.975', '1177.0', '58961', '1317.99991989136', '143.000520944595', '1174.99939894676', '1317.99991989136', '112', '0.0', '2648.0', '0.0', '1321.6', '1177.0', '2', '35.77962647', '0', 'N', '206.82290225', '3', '40', '40', '00035905040', '35905', 'Swift', '1', '3.18.11', '280.396874436814', '1.0', 'Hea_27Jul2015_V6.17_Swift_Rel4.5(Bld34)_27Jul2015_SDCpatch_16', '58951.0475', '35905', 'N', '96.759', '0.0', '0.0', '0.0', '273.109', '0.0', '96.781', '96.781', '193.771', '387.774', '0.0', '0.0', '0.0', '1170.91', '0.0', '0.0', '2.53090296146628'),
 ('266234', 'OJ287', '00035905041', '133.65865', '20.08717', '58954.249', '58964.0', '1396.572', '1368.105', '1402.0', '58965', '0.0', '98.0000410079956', '1399.99929904938', '1497.99934005737', '112', '0.0', '3000.0', '0.0', '1502.4', '1402.0', '2', '35.77392887', '0', 'N', '206.81946574', '4', '41', '41', '00035905041', '35905', 'Swift', '1', '3.1

In [57]:
sorted_table = table[np.argsort(table['xrt_expo_pc'])[::-1]]
sorted_table

__row,name,obsid,ra,dec,start_time,processing_date,xrt_exposure,uvot_exposure,bat_exposure,archive_date,af_insaa,af_inslew,af_onsource,af_total,att_flag,bat_expo_ev,bat_expo_mt,bat_expo_pl,bat_expo_rt,bat_expo_sv,bat_no_masktag,bii,cycle,grb_flag,lii,num_processed,obs_segment,orig_obs_segment,orig_obsid,orig_target_id,pi,prnb,processing_version,roll_angle,saa_fraction,software_version,stop_time,target_id,tdrss_flag,uvot_expo_bb,uvot_expo_bl,uvot_expo_gu,uvot_expo_gv,uvot_expo_m2,uvot_expo_mg,uvot_expo_uu,uvot_expo_vv,uvot_expo_w1,uvot_expo_w2,uvot_expo_wh,xrt_expo_im,xrt_expo_lr,xrt_expo_pc,xrt_expo_pu,xrt_expo_wt,Search_Offset
,,,deg,deg,d,d,s,s,s,d,s,s,s,s,,s,s,s,s,s,,deg,,,deg,,,,,,,,,deg,,,d,,,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,
object,object,object,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,int16,float64,int16,object,float64,int16,int32,int32,object,int32,object,int32,object,float64,float64,object,float64,int32,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
266998,OJ287,00030901053,133.68364,20.13617,55129.7771,57504.0,5052.15200,4957.88600,5162.00000,55140,0.00000,869.39585,5079.99543,5949.39128,112,0.00000,11560.00000,0.00000,5966.40000,5162.00000,2,35.81244,0,N,206.77223,9,53,53,00030901053,30901,Swift,1,3.17.03,116.29254,0.00000,Hea_27Jul2015_V6.17_Swift_Rel4.5(Bld34)_27Jul2015_SDCpatch_1,55130.0438,30901,N,343.11300,0.00000,0.00000,0.00000,947.71700,0.00000,431.56400,343.09100,1521.29600,1371.10500,0.00000,0.00000,0.00000,5052.15200,0.00000,0.00000,2.0059
266659,OJ287,00030901056,133.73066,20.11365,55153.3938,57505.0,5044.75900,4893.12200,5142.00000,55164,0.00000,982.20017,5064.99705,6047.19722,112,0.00000,11152.00000,0.00000,6062.40000,5142.00000,3,35.84652,0,N,206.81662,8,56,56,00030901056,30901,Swift,1,3.17.03,101.41411,0.00000,Hea_27Jul2015_V6.17_Swift_Rel4.5(Bld34)_27Jul2015_SDCpatch_1,55153.9073,30901,N,410.64500,0.00000,0.00000,0.00000,1191.97500,0.00000,410.65600,410.57900,822.62600,1646.64100,0.00000,0.00000,0.00000,5044.65500,0.00000,0.10400,1.5530
266477,OJ287,00030901044,133.68919,20.10498,54863.936,57380.0,4962.49600,4807.65100,5087.00000,54874,0.00000,915.00039,4989.99555,5904.99594,112,0.00000,8280.00000,0.00000,5924.80000,5087.00000,5,35.80692,0,N,206.81061,12,44,44,00030901044,30901,Swift,1,3.17.01,286.62843,0.00000,Hea_27Jul2015_V6.17_Swift_Rel4.5(Bld34)_27Jul2015_SDCpatch_0,54864.3458,30901,N,527.38500,0.00000,0.00000,0.00000,98.28100,0.00000,527.42900,484.26800,1056.35800,2113.93000,0.00000,0.00000,0.00000,4938.00100,0.00000,24.49500,0.8416
266687,OJ287,00030901055,133.72585,20.11459,55133.9993,57504.0,4894.21500,5014.92500,5182.00000,55144,5629.99756,499.99998,5129.99758,5629.99756,112,0.00000,8496.00000,0.00000,5640.00000,5182.00000,2,35.84258,0,N,206.81366,7,55,55,00030901055,30901,Swift,1,3.17.03,103.59186,1.00000,Hea_27Jul2015_V6.17_Swift_Rel4.5(Bld34)_27Jul2015_SDCpatch_1,55134.2322,30901,N,422.09700,0.00000,0.00000,0.00000,1212.31900,0.00000,422.19700,422.09700,845.10000,1691.11500,0.00000,0.00000,0.00000,4894.21500,0.00000,0.00000,1.3033
266744,OJ287,00030901054,133.72134,20.11656,55132.4514,57504.0,4857.28600,4762.72400,4928.00000,55143,0.00000,394.60048,4864.99738,5259.59786,112,0.00000,8712.00000,0.00000,5275.20000,4928.00000,2,35.83924,0,N,206.80962,6,54,54,00030901054,30901,Swift,1,3.17.03,100.19550,0.00000,Hea_27Jul2015_V6.17_Swift_Rel4.5(Bld34)_27Jul2015_SDCpatch_1,55132.6991,30901,N,402.11700,0.00000,0.00000,0.00000,1140.27400,0.00000,402.09500,402.09500,805.05000,1611.09300,0.00000,0.00000,0.00000,4857.28600,0.00000,0.00000,1.1074
266431,OJ287,00035011057,133.67183,20.10198,60096.0893,60106.0,4311.63900,4189.10100,4326.00000,60107,0.00000,760.42686,4324.97040,5085.39726,112,0.00000,8248.00000,0.00000,5099.20000,4326.00000,2,35.79055,0,N,206.80738,19,57,57,00035011057,35011,Swift,1,3.19.01,294.

In [58]:
obs_to_explore = [res for res in search_result if res['xrt_exposure'] > 4800]
obs_to_explore

[('266477', 'OJ287', '00030901044', '133.68919', '20.10498', '54863.936', '57380.0', '4962.496', '4807.651', '5087.0', '54874', '0.0', '915.000387042761', '4989.99555292726', '5904.99593997002', '112', '0.0', '8280.0', '0.0', '5924.8', '5087.0', '5', '35.80691847', '0', 'N', '206.81061475', '12', '44', '44', '00030901044', '30901', 'Swift', '1', '3.17.01', '286.62842607863', '0.0', 'Hea_27Jul2015_V6.17_Swift_Rel4.5(Bld34)_27Jul2015_SDCpatch_0', '54864.3458', '30901', 'N', '527.385', '0.0', '0.0', '0.0', '98.281', '0.0', '527.429', '484.268', '1056.358', '2113.93', '0.0', '0.0', '0.0', '4938.001', '0.0', '24.495', '0.8415688209222649'),
 ('266659', 'OJ287', '00030901056', '133.73066', '20.11365', '55153.3938', '57505.0', '5044.759', '4893.122', '5142.0', '55164', '0.0', '982.200165987015', '5064.99705404043', '6047.19722002745', '112', '0.0', '11152.0', '0.0', '6062.4', '5142.0', '3', '35.84651705', '0', 'N', '206.81661881', '8', '56', '56', '00030901056', '30901', 'Swift', '1', '3.17.0

In [59]:
obs = obs_to_explore[-1]
obs

('266998', 'OJ287', '00030901053', '133.68364', '20.13617', '55129.7771', '57504.0', '5052.152', '4957.886', '5162.0', '55140', '0.0', '869.395846068859', '5079.99543386698', '5949.39127993584', '112', '0.0', '11560.0', '0.0', '5966.4', '5162.0', '2', '35.81244061', '0', 'N', '206.7722297', '9', '53', '53', '00030901053', '30901', 'Swift', '1', '3.17.03', '116.292539099009', '0.0', 'Hea_27Jul2015_V6.17_Swift_Rel4.5(Bld34)_27Jul2015_SDCpatch_1', '55130.0438', '30901', 'N', '343.113', '0.0', '0.0', '0.0', '947.717', '0.0', '431.564', '343.091', '1521.296', '1371.105', '0.0', '0.0', '0.0', '5052.152', '0.0', '0.0', '2.0058891023723406')

In [68]:
obs = obs_to_explore[2]
obs
dlink = obs.getdatalink()

# only 3 summary columns are printed
dlink.to_table()[['ID', 'access_url', 'content_type']]

ID,access_url,content_type
object,object,object
ivo://nasa.heasarc/swiftmastr?00030901055,https://heasarc.gsfc.nasa.gov/xamin/query?table=swiftxrlog&constraint=obsid='00030901055',text/html
ivo://nasa.heasarc/swiftmastr?00030901055,https://heasarc.gsfc.nasa.gov/xamin/bib?table=swiftmastr&id=30901,text/html
ivo://nasa.heasarc/swiftmastr?00030901055,https://heasarc.gsfc.nasa.gov/xamin/query?table=swiftuvlog&constraint=obsid='00030901055',text/html
ivo://nasa.heasarc/swiftmastr?00030901055,https://heasarc.gsfc.nasa.gov/xamin/query?table=swiftbalog&constraint=obsid='00030901055',text/html
ivo://nasa.heasarc/swiftmastr?00030901055,https://heasarc.gsfc.nasa.gov/xamin/vo/datalink?datalink_key&id=ivo://nasa.heasarc/swiftmastr?00030901055/swift.obs,application/x-votable+xml;content=datalink
ivo://nasa.heasarc/swiftmastr?00030901055,https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/2009_10//00030901055/,directory


In [69]:
# loop through the observations
links = []
dlink_to_dir = [dl for dl in dlink if dl['content_type'] == 'directory']
link = dlink_to_dir[0]['access_url']
links.append(link)

In [70]:
on_sciserver = os.environ['HOME'].split('/')[-1] == 'idies'

if on_sciserver:
    # copy data locally on sciserver
    for link in links:
        os.system(f"cp -r /FTP/{link.split('FTP')[1]} .")

else:
    # use wget to download the data
    wget_cmd = ("wget -q -nH --no-check-certificate --no-parent --cut-dirs=6 -r -l0 -c -N -np -R 'index*'"
                " -erobots=off --retr-symlinks {}")

    for link in links:
        os.system(wget_cmd.format(link))